# Machine Learning: Text Classification Assignment

In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [2]:
path = 'AP_News/'
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+)/.*'
corpus = CategorizedPlaintextCorpusReader(path, DOC_PATTERN, cat_pattern=CAT_PATTERN)
corpus.fileids()

['health/http-apnews-com-03bc406312384416843138b2b23dec14.txt',
 'health/http-apnews-com-063eecb9a73e43b5a47f24d5d072de89.txt',
 'health/http-apnews-com-07145c801cc64b9c9fde6150af0e79db.txt',
 'health/http-apnews-com-08e5c195bf04471e9c4a127abe831d91.txt',
 'health/http-apnews-com-0dc745e5b66a47328b0be32f3cc9b1a2.txt',
 'health/http-apnews-com-145566f8c16a4d26ac36a317bbd3a02d.txt',
 'health/http-apnews-com-17d84e14096b4647828bf07cea2f6656.txt',
 'health/http-apnews-com-1a65f77ea55c4576ab9a981ae7c65ec5.txt',
 'health/http-apnews-com-1e96d18f8caa454e99d3f489865f8ff7.txt',
 'health/http-apnews-com-262de117d42947649c1a1caa6f4f70e7.txt',
 'health/http-apnews-com-3556845f3ab74186a26ec6d10739f9ca.txt',
 'health/http-apnews-com-386cc4805cfc49098181d525c113b65b.txt',
 'health/http-apnews-com-3bde9035af4044d082859ab587cfba0b.txt',
 'health/http-apnews-com-48e4b58e0a2e401cb52c5b600255657b.txt',
 'health/http-apnews-com-49877aba863e4f5199d0a22d68966bcc.txt',
 'health/http-apnews-com-5276e79e3d8f4ee

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [3]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]
docs[0]

'HONOLULU (AP) — The University of Hawaii seeks additional funds for student mental health services, scholarships and other items in a new supplementary budget request, officials said. The Board of Regents approved the fiscal year 2020-2021 supplemental operating budget of about $28 million Thursday, The Honolulu Star-Advertiser reported. The request will be submitted to the state Legislature and Democratic Gov. The university requested $2.6 million to hire 19 psychologists for the 10-campus system. The University of Hawaii Manoa has eight psychologists, the Hilo campus has three, and the West Oahu campus has 1.75 positions, while each community college has one position, said Allyson Tanouye, who coordinates mental health throughout the university system. "The national standard is one mental health professional per 1,000 to 1,500 students," Tanouye said. "If we add the 19 positions we will be up to one per 1,500. That\'s how low we are". The mental health funding would also expand prog

In [4]:
corpus.categories(corpus.fileids()[0])[0]

'health'

In [5]:
categories = [corpus.categories(fileid)[0] for fileid in corpus.fileids()]
categories

['health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [6]:
def preprocess(docs):
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('english')
    preprocessed = []

    for doc in docs:
        tokenized = word_tokenize(doc)
        cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower())) 
                   for token in tokenized 
                   if token.lower() not in stopwords.words('english')
                  if token.isalpha()]
        untokenized = " ".join(cleaned)
        preprocessed.append(untokenized)
    
    return preprocessed

    

In [7]:
preprocessed = preprocess(docs)
preprocessed[0]

'honolulu ap univers hawaii seek addit fund student mental health servic scholarship item new supplementari budget request offici said board regent approv fiscal year supplement oper budget million thursday honolulu report request submit state legislatur democrat gov univers request million hire psychologist system univers hawaii manoa eight psychologist hilo campus three west oahu campus posit communiti colleg one posit said allyson tanouy coordin mental health throughout univers system nation standard one mental health profession per student tanouy said add posit one per low mental health fund would also expand program prevent suicid reduc mental health stigma provid peer educ alert new student parent colleg transit challeng offici said largest item supplement budget million expand hawaii promis program scholarship state institut univers propos flat amount cover tuition fee hawaii resid qualifi feder pell grant look focus needi student go campus said donald straney vice presid academ

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [8]:
print(len(categories))
len(preprocessed)

217


217

In [9]:
X_train, X_test, y_train, y_test = tts(preprocessed, categories, test_size=0.3)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
rfc = RandomForestClassifier(n_estimators=100)
pipe = Pipeline([
    ('vect', tfidf),
    ('clf', rfc)
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('vect', TfidfVectorizer()), ('clf', RandomForestClassifier())])

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [11]:
y_pred_test = pipe.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

      health       0.74      0.82      0.78        17
    politics       0.80      0.71      0.75        17
      sports       0.88      0.93      0.90        15
        tech       0.69      0.65      0.67        17

    accuracy                           0.77        66
   macro avg       0.77      0.78      0.77        66
weighted avg       0.77      0.77      0.77        66



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [12]:
scores = cross_val_score(pipe, preprocessed, categories, cv=10, scoring='f1_macro')
scores

array([0.86752137, 0.81742424, 0.83791209, 0.81730769, 0.91666667,
       0.78030303, 0.66813725, 0.7534188 , 0.73023504, 0.71190476])

In [13]:
scores.mean()

0.7900830950095656

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [14]:
pipe.fit(preprocessed, categories)

Pipeline(steps=[('vect', TfidfVectorizer()), ('clf', RandomForestClassifier())])

In [15]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'

In [16]:
import requests
from bs4 import BeautifulSoup

result = requests.get(url)
soup = BeautifulSoup(result.text)
doc = soup.find('section', attrs={'name':'articleBody'}).text
preprocessed = preprocess([doc])
pipe.predict(preprocessed)

array(['tech'], dtype='<U8')